In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [2]:
import itertools
from operator import itemgetter
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from cvxopt import matrix, solvers, spmatrix

# Import linear demand curve coefficients and initialized paratemers 
from initialize import initialize
from genregparam import linparams

np.set_printoptions(precision=2, suppress=True)

In [3]:
# Parameters for simulation study
n_class = 2
los = 3
capacity = 50
intensity = 1.5
slopes_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

# Total combinations of arrivals
combs = n_class * 7 * los

In [4]:
# Partitioned protection levels, nested protection levels, representative revenue, and
# discount ration for each virtual bucket, each stay night
buckets, pl_prt, pl, rates_vir, ratios = initialize(capacity, intensity, rates_init)
slopes, intercepts = linparams(capacity, intensity, slopes_init, rates_init)

In [5]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [6]:
# Generate mean demand for poisson arrivals for each rate, arrival day, and los combination
mus = intercepts + slopes * rates_arrival_los
mus = np.round(mus, 0)

In [7]:
# Flatten arrays for quadratic programming formulation
slopes_flat = slopes.reshape(n_class * 7 * los)
intercepts_flat = intercepts.reshape(n_class * 7 * los)

In [8]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combinations.
# Column indexes 0-20 are associated with decision variables for rate class 1
# Column indexes 21-41 are associated with decision variables for rate class 2
G = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
# Arrivals that span Sunday stay night for rate class 1
G[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 1
G[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
# Arrivals that span Sunday stay night for rate class 2
G[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 2
G[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

h1 = intercepts_flat * G

In [9]:
# Here, be careful. For the capacity constraints, LHS is sum of demands that span
# a specific stay night in question. But decision variables for quadratic programming
# are rates. 
# demand1 + demand2 + demand3 = intercept1 + slope1 * rate1 + intercept2 + slope2 * rate2 + 
# intercept3 + slope3 * rate3 <= capacity
# --> slope1*rate1 + slope2*rate2 + slope3*rate3 <= capacity - (intercept1+intercept2+intercept3)
h1 = np.sum(h1, axis=1)
# G for capacity constraints, Negative identity matrix for non-negativity
G = slopes_flat * G
G = np.concatenate((G, -np.identity(combs)), axis=0)
# Inequality equations, RHS
# First h for capacity rhs and second component for non-negativity rhs.
h = np.concatenate((capacity * np.ones(7) - h1, np.zeros(combs)), axis=0)

In [10]:
# This part is a little bit tedious, but I couldn't find an elegant way of doing it
# Purpose of this section is to make sure optimal rates for e.g., Monday arrival two-night 
# stay is equal to optimal rate for Monday arrival one-night stay and Tuesday arrival one-
# night stay. This is how the rates are calculated in hotel industry for multiple nights stay.
# It is different from airline pricing with multiple legs.
arr1 = [3*i for i in range(14)]
arr2 = [(3*i) for i in range(1, 7)] + [0] + [(3*i) % 42 for i in range(8, 14)] + [21]
arr3 = [3*(i+2) % 21 for i in range(7)] + [3*(i+2) for i in range(7, 12)] + [21] + [24]
arr4 = np.concatenate((np.arange(1, 41, 3), np.arange(2, 42, 3))).tolist()
els = np.concatenate((np.repeat(1.0, 70), np.repeat(-1, 28)))
A = spmatrix(els, np.concatenate((range(28), range(28), range(14, 28), range(28))).tolist(), 
             arr1 + arr1 + arr2 + arr2 + arr3 + arr4)
b = matrix(np.zeros(28))

In [11]:
# Quadratic programming
#                  minimize (1/2)x_T*Q*x + p_T*x
#                  subject to G*x <= h
#                             A*x = b
slopes_diag = np.diag(slopes_flat)
Q = 2 * matrix(-slopes_diag)
p = matrix(-intercepts_flat)

# Convert numpy arrays to cvxopt matrix forms
G = matrix(G)
h = matrix(h)

In [12]:
# Solve quadratic programming
sol = solvers.qp(Q, p, G, h, A, b)
print(sol)

{'x': <42x1 matrix, tc='d'>, 'y': <28x1 matrix, tc='d'>, 's': <49x1 matrix, tc='d'>, 'z': <49x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 0.02219480980317204, 'relative gap': 4.605552276806852e-07, 'primal objective': -48191.4186816272, 'dual objective': -48191.440876437024, 'primal infeasibility': 1.3101760682293512e-13, 'dual infeasibility': 2.8391552314940973e-16, 'primal slack': 2.741605610475992e-06, 'dual slack': 7.275662333109687e-07, 'iterations': 5}


In [13]:
rates_DP = np.array(sol['x']).reshape(n_class, 7, los)
rates_DP = np.round(rates_DP, 0)

In [14]:
rates_DP

array([[[105., 252., 405.],
        [147., 300., 437.],
        [153., 289., 428.],
        [136., 275., 385.],
        [138., 249., 383.],
        [111., 245., 350.],
        [134., 239., 386.]],

       [[102., 235., 372.],
        [133., 271., 396.],
        [138., 263., 391.],
        [125., 253., 358.],
        [128., 233., 357.],
        [105., 229., 331.],
        [124., 226., 359.]]])

In [15]:
# Initialize rates for adaptive pricing algorithm and first-come, first-serve approach
rates_AP = rates_arrival_los
rates_FCFS = rates_arrival_los
# DAF indicates order of rates as dynamic pricing rates, adaptive pricing rates, and FCFS rates
rates_DAF = [rates_DP, rates_AP, rates_FCFS]

# Generate nonhomogenous Poisson process true bookings (i.e., customer demand) on a weekly basis
# the mean of the Poisson process will be equal to the linear demand curve function of the room rates
mus_DAF = [np.maximum(intercepts + slopes * rates, 0) for rates in rates_DAF]

demand_DAF = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus_DAF]
demand_DAF = [np.array(x).reshape(n_class, 7, los) for x in demand_DAF]

In [16]:
# Consider first week, Sunday stay night
buckets_sun = [[rda for rda in rdas if rda[1] == 0] for rdas in buckets[0]]
capacity_left = [capacity] * 3
sold_DAF = []
for rdas in reversed(buckets_sun):
    sold_bucket = []
    for rda in rdas:
        sold = [min(demand_DAF[i][rda], capacity_left[i]) for i in range(len(demand_DAF))]
        revenue = [rates_DAF[i][rda] * sold[i] for i in range(len(demand_DAF))]
        sold_bucket.append((rda, sold, revenue))
        capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
    sold_DAF.append(sold_bucket)
    
# Remove empty lists
sold_DAF = list(filter(None, sold_DAF))

In [17]:
sold_DAF

[[((0, 0, 1), [9, 8, 7], [2268.0, 2160, 1890]),
  ((0, 0, 0), [14, 8, 11], [1470.0, 1080, 1485]),
  ((1, 0, 0), [13, 9, 15], [1326.0, 1035, 1725]),
  ((1, 0, 1), [3, 5, 6], [705.0, 1150, 1380]),
  ((1, 0, 2), [6, 6, 6], [2232.0, 2070, 2070])],
 [((0, 0, 2), [4, 4, 5], [1620.0, 1620, 2025])]]